<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/05_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Lab 05: Notebook Testing com nbval

**Framework:** Scientific Validation Hub
**Ferramenta:** [nbval (Pytest Plugin)](https://github.com/computationalmodelling/nbval)
**Objetivo:** Tratar Jupyter Notebooks como código de produção, aplicando testes unitários automatizados para garantir que os resultados são reprodutíveis em qualquer máquina.

---

### ⚡ Quick Start (Modo Assistido)
Neste laboratório, vamos criar um notebook "Cobaia" e soltar um auditor automatizado em cima dele.

1.  **Simule:** Rode as células para gerar um notebook de experimento fake (`experimento_cobaia.ipynb`).
2.  **Teste:** O script rodará o `pytest` para verificar se o código da cobaia funciona como prometido.
3.  **Audite:** Abra o Assistente Gemini e cole:

> "Atue como Engenheiro de QA (Quality Assurance).
> Analise os logs do Pytest abaixo.
> 1. Quantos testes foram coletados e quantos passaram?
> 2. O que significa o status 'passed' no contexto de reprodutibilidade científica?
> 3. Se aprovado, gere um badge Markdown verde: 'Notebook Integrity: VERIFIED'."

In [ ]:
# Instala o framework de testes (pytest) e o plugin de notebooks (nbval)
!pip install pytest nbval -q
print("✅ Framework de Testes instalado.")

In [ ]:
import nbformat as nbf

# 1. Vamos criar um notebook programaticamente para ser testado
# Isso simula o notebook que um pesquisador enviaria para revisão
nb = nbf.v4.new_notebook()

text_cell = nbf.v4.new_markdown_cell("# Experimento de Cálculo de Densidade")

# Célula de Código 1: Uma operação determinística simples
code_cell_1 = nbf.v4.new_code_cell("print('Iniciando Validação...')")
# IMPORTANTE: No nbval, nós salvamos o output ESPERADO.
# Se o código rodar e gerar algo diferente disso, o teste falha.
code_cell_1.outputs.append(nbf.v4.new_output("stream", text="Iniciando Validação...\n"))

# Célula de Código 2: Um cálculo matemático
code_cell_2 = nbf.v4.new_code_cell("x = 10\ny = 5\nprint(f'Resultado: {x * y}')")
code_cell_2.outputs.append(nbf.v4.new_output("stream", text="Resultado: 50\n"))

# Adicionando células ao notebook objeto
nb.cells = [text_cell, code_cell_1, code_cell_2]

# Salvando o arquivo no disco do Colab
filename = "experimento_cobaia.ipynb"
with open(filename, 'w') as f:
    nbf.write(nb, f)

print(f"📄 Notebook Cobaia criado: {filename}")
print("   Este arquivo contém código e o output esperado (Resultado: 50).")
print("   Agora vamos ver se ele reproduz corretamente...")

In [ ]:
import pytest
import sys

# 2. Rodando o Pytest
# O flag --nbval-lax diz: "Rode o código e veja se não quebra".
# O flag --nbval diz: "Rode e veja se o output é IDÊNTICO ao salvo".
print("🔍 EXECUTANDO AUDITORIA AUTOMATIZADA (nbval)...\n")
print("="*60)

# Executando comando de shell para visualizar o output colorido do pytest
!pytest --nbval {filename} -v

print("="*60)
print("\nLEGENDA DO RESULTADO:")
print("💚 PASSED: O código rodou e reproduziu exatamente o resultado salvo.")
print("🔴 FAILED: O código quebrou ou gerou um resultado diferente (Não reprodutível).")

In [ ]:
# 3. Vamos sabotar o experimento para ver o erro acontecer
print("\n😈 MODO SABOTAGEM: Alterando o código sem atualizar o output...")

# Alteramos a lógica (10 * 5 agora vira 10 * 100), mas o output esperado continua 50
nb.cells[2].source = "x = 10\ny = 100\nprint(f'Resultado: {x * y}')"

with open("experimento_falho.ipynb", 'w') as f:
    nbf.write(nb, f)

print("🔍 Rodando teste no notebook sabotado...")
!pytest --nbval experimento_falho.ipynb